In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
import matplotlib
import matplotlib.pyplot as plt
import scipy.optimize as optimize
import Estimate
import figure 
from EconModel import cpptools
import pickle

from LimitedCommitmentModel import LimitedCommitmentModelClass

# plot style
linestyles = ['-','--','-.',':',':']
markers = ['o','s','D','*','P']
linewidth = 2
font_size = 17
font = {'size':font_size}
matplotlib.rc('font', **font)


plt.rcParams.update({'figure.max_open_warning': 0,'text.usetex': False})
path = 'output/'

SAVE = False

# c++ settings

do_compile = True
threads = 20
T = 20
#cpptools.setup_nlopt(folder='cppfuncs/', do_print = True) #install nlopt

import warnings
warnings.filterwarnings("ignore")

#samme antal observationer: KØR WAGE TYPE; LOVEKONTROL; ETC
#norm1 N = 9,000    model FC  
#norm1 N = 2,400    model NC 
#norm1 N = 2,100    model LC 




#NEW 9000
#model LC 2700      6000
#model NC 3000      6400
#model FC 9200      21000


# restr feas 20000 (prøv at kør det med 9000 bagefter) 

#rest feas 9000
#model_FC 9200 --> 6200
#model LC 2700  --> 6800
#model NC 3000 --> 7800  

#rest nonfeas 9000
#model_FC 9200 -->  6200
#model LC 2700  -->  489... kan ikke køre
#model_LC 12000 -->  2400
#model NC 3000 --> 6800  

#ALMINDELIGE 9000
#model_FC 14,000
#model_LC 3500
#model_NC 3500 

#REST FEAS 9000
#model_FC 30,000  Kørt
#model_LC 9000  
#model_NC 9000 

#REST NONFEAS  9000
#model_FC           drop 
#model_LC 35000     kørt
#model_NC 9000   kørt


#REST FEAS 9000 med med 0.05... 
#model_FC   350,000
#model_LC 125,000
#model_NC  125,000 kørt




# Load the model

In [2]:
name = 'model_LC'
model_name = "model_LC"
do_distr_yerror = "none"
do_yerror = 0
#do_BMI_dummy = False 
do_sep_shock = False 
#name_error = f'{do_distr_yerror}{do_yerror}std_Yerror_rest0_restOutl'
name_error = f'{do_distr_yerror}{do_yerror}std_Yerror_restr_feas_small_threshold'
N = 125000
NN = "NN" #NN er samme antal observationer, N er samme antal simuleringer
NNO = 9000
rest_sample_feas = True
rest_sample_nonfeas    = False
#name_error = "uni2std_Yerror"

specs = {}
#specs.update({f'test2':{'latexname':'limited', 'par':{ 'T':T, 'num_K': 5, 'num_A': 10 ,  'do_HK': False, 'threads':threads,'bargaining':0}}})


specs.update({f'model_LC':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': False, 'simN': N,  'threads':threads,'bargaining':1}}})
#specs.update({f'model_NC':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': False, 'simN': N,  'threads':threads,'bargaining':2}}})
#specs.update({f'model_FC':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': False, 'simN': N,  'threads':threads,'bargaining':0}}})

# solve different models
models = {}
for m,(name,spec) in enumerate(specs.items()):
    
    # setup model
    model = LimitedCommitmentModelClass(name=name,par=spec['par'])
    model.spec = spec

    compile_now = True if do_compile & (m==0) else False
    model.link_to_cpp(force_compile=compile_now)
    

# setup model
#model = LimitedCommitmentModelClass(name=name)    
#model.link_to_cpp(force_compile=True)


#load par and sol
pickle_file = f'output/{name}_sol.pickle'

with open(pickle_file, 'rb') as f:
    model.sol = pickle.load(f)

pickle_file = f'output/{name}_par.pickle'

with open(pickle_file, 'rb') as f:
    model.par = pickle.load(f)
    model.par.simN = N

In [3]:
#Do 1 time estimate
data = Estimate.create_data(model,start_p = 2, end_p = model.par.T-2, yerror = do_distr_yerror, scale_st = do_yerror )

data = Estimate.create_variable(data, model.par)

if rest_sample_nonfeas:  
    if model_name != "model_FC":
        # Remove those who do not update bargaining weight
        
        data['delta_log_barganing'] = data['delta_log_barganing'].mask(data['delta_log_barganing'].between(-0.001,0.001))
    

if rest_sample_feas:  
    #not at the limit
    data =data[data['omega_res_w'] < 0.05]
    data =data[data['omega_res_w'] > -0.05]
    data =data[data['omega_res_m'] < 0.05]
    data =data[data['omega_res_m'] > -0.05]

    

    data =data[data['omega_res_w_l'] < 0.05]
    data =data[data['omega_res_w_l'] > -0.05]
    data =data[data['omega_res_m_l'] < 0.05]
    data =data[data['omega_res_m_l'] > -0.05]

        
    data =data[data['omega_res_w_l2'] < 0.05]
    data =data[data['omega_res_w_l2'] > -0.05]
    data =data[data['omega_res_m_l2'] < 0.05]
    data =data[data['omega_res_m_l2'] > -0.05]

print(data)


_, wald_FC_w , wald_NC_w, _,_,  _ = Estimate.main_est(data,gender= "w", do_estimate_wage = "est_omega", print_reg = True, shadow_value_simple=2, do_control_love = False, part_earning_simple =2, control_cons = 2, wealth_love = 1)


           idx   t    wealth   hours_w   hours_m      cons    wage_w  \
57          14  15  1.520553  0.707214  1.000000  1.527298  1.649244   
70          18  11  1.539428  0.906394  0.858375  1.131354  1.434266   
102         28  10  3.483959  0.838378  0.847785  1.151450  1.436612   
107         28  15  3.279879  0.822350  0.791745  1.767415  1.564967   
113         29  13  3.078506  0.881873  0.826546  1.524428  1.655117   
...        ...  ..       ...       ...       ...       ...       ...   
510548  124954  14  2.323006  0.953020  0.744687  1.608106  1.595996   
510638  124972  14  1.846936  0.982886  0.749112  1.611467  1.583185   
510639  124972  15  1.868194  0.988504  0.702753  1.621601  1.607848   
510712  124992  12  1.232509  0.946273  0.959448  1.585375  1.500761   
510720  124995  12  1.925192  0.874629  0.985995  1.326237  1.636531   

          wage_m  earnings_w  earnings_m  ...  delta_omega_w_l2  \
57      1.701262    1.166369    1.701262  ...          0.021925   
7

In [4]:
model.par.seed = 2024
np.random.seed(model.par.seed)

no_sim = 100
wald_FC_w_pval = np.nan + np.ones((4,2,2,2,2,2,no_sim) )
wald_NC_w_pval = np.nan + np.ones((4,2,2,2,2,2,no_sim) )
wald_FC_m_pval = np.nan + np.ones((4,2,2,2,2,2,no_sim) )
wald_NC_m_pval = np.nan + np.ones((4,2,2,2,2,2,no_sim) )




coef_w = {}
coef_m = {}





N_w= np.nan + np.ones(no_sim) 
N_m= np.nan + np.ones(no_sim) 
#loop over shadow values
#for j in range(4):
#    j = j +1
for k in [1,2,3,4]:
    if k == 1:
        #BASELINE WITH DETAILED SHADOW
        j = 2 #nonpar shadow
        jj = 2 #nonpar controls
        jjj = 1 #par wage
        p = False #True bargaining
        ip = int(p)
        ik = 0 # wage
        do_wage = "est_omega"
        im = 0 #love
        do_love = False 
        z = False #do Heckman selection
        iz = int(z)
    if k == 2:
        #include HECKMANS selection 
        j = 2 #nonpar shadow
        jj = 2 #nonpar controls
        #jjj = 1 #par wage
        p = False #True bargaining
        ip = int(p)
        ik = 0 # wage
        do_wage = "est_omega"
        im = 0 #love
        do_love = False 
        z = True  #do Heckman selection
        iz = int(z)
    if k == 3:
        #include HECKMANS selection + love + true wage shock 
        j = 2 #nonpar shadow
        jj = 2 #nonpar controls
        #jjj = 1 #par wage
        p = False #True bargaining
        ip = int(p)
        ik = 1 # wage
        do_wage = "true_omega"
        im = 1 #love
        do_love = True 
        z = True  #do Heckman selection
        iz = int(z)
    if k == 4:
        #include HECKMANS selection + love + true wage shock 
        j = 2 #nonpar shadow
        jj = 2 #nonpar controls
        #jjj = 1 #par wage
        p = True #True bargaining
        ip = int(p)
        ik = 1 # wage
        do_wage = "true_omega"
        im = 1 #love
        do_love = True 
        z = True  #do Heckman selection
        iz = int(z)

    for i in range(no_sim):
        print(f'k = {k}, i = {i}')
        data = Estimate.create_data(model,start_p = 2, end_p = model.par.T-2, yerror = do_distr_yerror, scale_st = do_yerror)
        data = Estimate.create_variable(data, model.par)
        if rest_sample_nonfeas:  
            if model_name != "model_FC":
                # Remove those who do not update bargaining weight
                #data = data[data['delta_log_barganing'] != 0]
                
                data['delta_log_barganing'] = data['delta_log_barganing'].mask(data['delta_log_barganing'].between(-0.001,0.001))
                

                #data =data[data['delta_log_barganing_l'] > -0.1]
                #data =data[data['delta_log_barganing_l'] <  0.1]

                #data =data[data['delta_log_barganing_l2'] > -0.1]
                #data =data[data['delta_log_barganing_l2'] <  0.1]

        if rest_sample_feas:  
            #not at the limit
            data =data[data['omega_res_w'] < 0.05]
            data =data[data['omega_res_w'] > -0.05]
            data =data[data['omega_res_m'] < 0.05]
            data =data[data['omega_res_m'] > -0.05]

            

            data =data[data['omega_res_w_l'] < 0.05]
            data =data[data['omega_res_w_l'] > -0.05]
            data =data[data['omega_res_m_l'] < 0.05]
            data =data[data['omega_res_m_l'] > -0.05]

                
            data =data[data['omega_res_w_l2'] < 0.05]
            data =data[data['omega_res_w_l2'] > -0.05]
            data =data[data['omega_res_m_l2'] < 0.05]
            data =data[data['omega_res_m_l2'] > -0.05]

       
        data = Estimate.aux_est(data,model.par)
        _, wald_FC_w , wald_NC_w, wald_FC_noW_w , wald_NC_noW_w, c_w = Estimate.main_est(data,gender= "w", do_estimate_wage = do_wage, print_reg = False, shadow_value_simple=j, do_control_love = do_love, part_earning_simple = jj, control_cons= jj,  do_true_barg = p, control_select = z)
        _, wald_FC_m , wald_NC_m, wald_FC_noW_m , wald_NC_noW_m, c_m = Estimate.main_est(data,gender= "m", do_estimate_wage = do_wage, print_reg = False, shadow_value_simple=j, do_control_love = do_love, part_earning_simple = jj, control_cons= jj,  do_true_barg = p, control_select = z)
        wald_FC_w_pval[j-1, jj-1,ip,ik, im, iz, i] = wald_FC_w.pvalue
        wald_FC_m_pval[j-1, jj-1,ip,ik, im, iz, i] = wald_FC_m.pvalue
        wald_NC_w_pval[j-1, jj-1,ip,ik, im, iz, i] = wald_NC_w.pvalue
        wald_NC_m_pval[j-1, jj-1,ip,ik, im, iz, i] = wald_NC_m.pvalue

        #wald_FC_noW_w_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_FC_noW_w.pvalue
        #wald_FC_noW_m_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_FC_noW_m.pvalue
        #wald_NC_noW_w_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_NC_noW_w.pvalue
        #wald_NC_noW_m_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_NC_noW_m.pvalue
        
        coef_w[f'shadow{j}_inc{jj}_cons{jj}_wage{ik}_love{im}_barg{ip}_select{iz}_sim{i}'] =  c_w
        coef_m[f'shadow{j}_inc{jj}_cons{jj}_wage{ik}_love{im}_barg{ip}_select{iz}_sim{i}'] =  c_m
        



k = 1, i = 0
k = 1, i = 1
k = 1, i = 2
k = 1, i = 3
k = 1, i = 4
k = 1, i = 5
k = 1, i = 6
k = 1, i = 7
k = 1, i = 8
k = 1, i = 9
k = 1, i = 10
k = 1, i = 11
k = 1, i = 12
k = 1, i = 13
k = 1, i = 14
k = 1, i = 15
k = 1, i = 16
k = 1, i = 17
k = 1, i = 18
k = 1, i = 19
k = 1, i = 20
k = 1, i = 21
k = 1, i = 22
k = 1, i = 23
k = 1, i = 24
k = 1, i = 25
k = 1, i = 26
k = 1, i = 27
k = 1, i = 28
k = 1, i = 29
k = 1, i = 30
k = 1, i = 31
k = 1, i = 32
k = 1, i = 33
k = 1, i = 34
k = 1, i = 35
k = 1, i = 36
k = 1, i = 37
k = 1, i = 38
k = 1, i = 39
k = 1, i = 40
k = 1, i = 41
k = 1, i = 42
k = 1, i = 43
k = 1, i = 44
k = 1, i = 45
k = 1, i = 46
k = 1, i = 47
k = 1, i = 48
k = 1, i = 49
k = 1, i = 50
k = 1, i = 51
k = 1, i = 52
k = 1, i = 53
k = 1, i = 54
k = 1, i = 55
k = 1, i = 56
k = 1, i = 57
k = 1, i = 58
k = 1, i = 59
k = 1, i = 60
k = 1, i = 61
k = 1, i = 62
k = 1, i = 63
k = 1, i = 64
k = 1, i = 65
k = 1, i = 66
k = 1, i = 67
k = 1, i = 68
k = 1, i = 69
k = 1, i = 70
k = 1, i = 71
k 

In [5]:
#SAVE PICKLE
pickle_file = f'output/TRUE{name}_w_{NN}{NNO}_S{no_sim}_{name_error}_testFC.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(wald_FC_w_pval,f)
pickle_file = f'output/TRUE{name}_m_{NN}{NNO}_S{no_sim}_{name_error}_testFC.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(wald_FC_m_pval,f)
pickle_file = f'output/TRUE{name}_w_{NN}{NNO}_S{no_sim}_{name_error}_testNC.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(wald_NC_w_pval,f)
pickle_file = f'output/TRUE{name}_m_{NN}{NNO}_S{no_sim}_{name_error}_testNC.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(wald_NC_m_pval,f)

pickle_file = f'output/TRUE{name}_m_{NN}{NNO}_S{no_sim}_{name_error}_coef.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(coef_m,f)

pickle_file = f'output/TRUE{name}_w_{NN}{NNO}_S{no_sim}_{name_error}_coef.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(coef_w,f)



NameError: name 'stop' is not defined

(4, 2, 2, 1, 1, 2, 100)


In [ ]:
stop